## Step 1: Preprocessing Data



### Step 1.1: Move to the right directory

In [ ]:
import os, re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from argparse import Namespace
import nltk
import glob
from collections import defaultdict
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Rouge Evaluation

Generate New Reference Trailer

In [ ]:
joker_files = glob.glob("./PreSumm/split_corpus/test_Joker_*.txt")
new_reference_trailer = []
for file in joker_files:
  lines = open(file).readlines()
  new_reference_trailer.extend([line for line in lines if "@highlight" in line])

In [ ]:
for movie in movie_names:
  print(movie)

In [ ]:
from rouge_metric import PyRouge
import random
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)
# hypothesis = open("./Joker_summary.txt").readlines()
hypothesis = [open("./bert-extractive-summarization/Joker_summary.txt").read()]
# references = open("./corpus_raw/DC/trailer_time/Joker.txt").readlines()
references = [[''.join(new_reference_trailer)]]
n = min(len(hypothesis), len(references))
scores = rouge.evaluate(hypothesis, references)
scores

{'rouge-1': {'f': 0.07887323943661972, 'p': 0.04590163934426229, 'r': 0.28},
 'rouge-2': {'f': 0.039660056657223795,
  'p': 0.023026315789473683,
  'r': 0.14285714285714285},
 'rouge-4': {'f': 0.022922636103151865,
  'p': 0.013245033112582781,
  'r': 0.0851063829787234},
 'rouge-l': {'f': 0.07887323943661972, 'p': 0.04590163934426229, 'r': 0.28},
 'rouge-s4': {'f': 0.02521489971346705,
  'p': 0.01456953642384106,
  'r': 0.09361702127659574},
 'rouge-su4': {'f': 0.034318398474737846,
  'p': 0.019845644983461964,
  'r': 0.1267605633802817},
 'rouge-w-1.2': {'f': 0.05440732550946698,
  'p': 0.0340523820160018,
  'r': 0.13525879526081125}}

In [ ]:
os.chdir('./stanford-corenlp-4.3.1/')

In [ ]:
%mkdir ./new_trailer_test

In [ ]:
%cd ../bert-extractive-summarization
from models.model_builder import ExtSummarizer
from ext_sum import summarize
import torch
model_type = 'distilbert' #@param ['bertbase', 'distilbert', 'mobilebert']
checkpoint = torch.load("../stanford-corenlp-4.3.1/models/model_step_5000.pt", map_location='cpu')["model"]
model = ExtSummarizer(checkpoint=checkpoint, bert_type=model_type, device="cpu")


# with open("../PreSumm/split_corpus/test_Batman Begins_0.txt", "w") as f:
#   f.write(processed_text)



In [ ]:
path ='../PreSumm/split_corpus/test_Ant-Man_0.txt'

In [ ]:
summary = summarize(path, result_fp, model, max_length=4)

In [ ]:
summary

"Ant-Man 0 Stark! 66 The Pym Particle is a miracle.\nAnt-Man 0 Stark! 66 The Pym Particle is a miracle. Please. 69 Don't let your past determine the future."

In [ ]:
summary.splitlines()

['Ant-Man 0 Stark! 66 The Pym Particle is a miracle.',
 "Ant-Man 0 Stark! 66 The Pym Particle is a miracle. Please. 69 Don't let your past determine the future."]

In [ ]:
open(path).readlines()

["Shazam 754 but you've run from foster homes in six counties, Mr. Batson.\n",
 '758 From good people\n',
 '761 who want you, all in pursuit of someone\n',
 '764 who arguably does not.\n',
 "767 It's time someone looked you in the eye and told it to you straight.\n",
 "771 I don't need parents to play make-believe with.\n",
 '774 - I got a mom. - Yes.\n',
 "777 You're some detective.\n",
 '779 Seventy-three Batsons\n',
 "781 between the ages of 28 to 40, and you've crossed out every one.\n",
 "784 She's still out there. I know it.\n",
 "796 There's a couple outside, Mr. Batson.\n",
 '799 They run a group home.\n',
 '802 I can take care of myself.\n',
 "803 When you're 18.\n",
 "805 Until then, I won't have you living on the streets,\n",
 '808 looking for someone who never looked for you.\n',
 '811 Give these people a chance.\n',
 "814 'Cause that's what they're giving you.\n",
 "816 You're out of options.\n",
 "829 I'm gonna skip the spiel.\n",
 '830 Just know that me and Rosa, we get 

In [ ]:
shazam_ant_man_paths = defaultdict(list) 
for file in glob.glob("../PreSumm/split_corpus/test_*"):
  if "test_Shazam" in file:
    shazam_ant_man_paths["Shazam"].append(file)
  elif "test_Ant" in file:
    shazam_ant_man_paths["Ant-Man"].append(file)
shazam_ant_man_paths

for test_movie in shazam_ant_man_paths:
  movie_paths = shazam_ant_man_paths[test_movie]
  i = 0
  summary_list = []
  length_arr = np.random.choice([0, 1], size=(len(movie_paths),))
  for j, path in enumerate(movie_paths):
    result_fp = f'./new_trailer_test/{test_movie}_{j}_summary.txt'
    if length_arr[j] == 0:
      continue
    summary = summarize(path, result_fp, model, max_length=1)
    if j % 10 == 0:
      print(f"Finished {test_movie} summary {j}")
      print(length_arr[j], summary)
    summary_list.append(summary)
  full_summary_path = f"./{test_movie}_summary_{i}.txt"
  with open(full_summary_path, "w") as f:
    f.writelines(summary_list)

Finished Ant-Man summary 0
1 Ant-Man 0 Stark!
Finished Ant-Man summary 20
1 Ant-Man 4262 That's witchcraft.


In [ ]:
pattern = ".+test_(.+)_\d+.txt"
test_files = glob.glob("../PreSumm/split_corpus/test_*.txt")
movie_names = set([re.match(pattern, path).groups()[0] for path in test_files])

In [ ]:

for movie in movie_names:
  for i, file_path in enumerate(test_files):
    if movie in file_path:
      input_fp = file_path
      result_fp = f'{movie}_summary{i}.txt'
      summary = summarize(input_fp, result_fp, model, max_length=1)
      print(f"{movie} summarization {i} finished")

Joker summarization 26 finished
Joker summarization 27 finished
Joker summarization 30 finished
Joker summarization 32 finished
Joker summarization 33 finished
Joker summarization 34 finished
Joker summarization 35 finished
Joker summarization 36 finished
Joker summarization 37 finished
Joker summarization 38 finished
Joker summarization 39 finished
Joker summarization 40 finished
Joker summarization 41 finished
Joker summarization 42 finished
Joker summarization 43 finished
Joker summarization 44 finished
Joker summarization 45 finished
Joker summarization 46 finished
Joker summarization 48 finished
Joker summarization 50 finished
Joker summarization 52 finished
Thor Ragnarok summarization 0 finished
Thor Ragnarok summarization 1 finished
Thor Ragnarok summarization 2 finished
Thor Ragnarok summarization 3 finished
Thor Ragnarok summarization 4 finished
Thor Ragnarok summarization 5 finished
Thor Ragnarok summarization 6 finished
Thor Ragnarok summarization 7 finished
Thor Ragnarok su

In [ ]:
summary_files = glob.glob("*_summary*.txt")
movie_summaries = defaultdict(list)

for file_name in summary_files:
  movie_name = file_name.split('_')[0]
  with open(file_name) as f:
    movie_summaries[movie_name].append(f.read())

movie_summaries

defaultdict(list,
            {'Ant-Man': ['Ant-Man 0 Stark!\n',
              'Ant-Man 332 Thank you.\n',
              'Ant-Man 218 Oh, yeah.\n',
              'Ant-Man 738 I am proud to present the end of warfare as we know it.\n',
              'Ant-Man 487 No, no, no.\n',
              "Ant-Man 916 I didn't know what time your party started.\n",
              "Ant-Man 1516 Well, here's the thing.\n",
              "Ant-Man 2020 I'm starting to think that you prefer the inside of a jail cell.\n",
              "Ant-Man 1239 It's the wrong details.\n",
              'Ant-Man 1020 Get an apartment, 1022 get a job, pay child support.\n',
              'Ant-Man 2257 Yank up to go up.\n',
              "Ant-Man 2430 which, judging by the fact that you're sitting opposite me, 2433 I take it that you're interested in.\n",
              'Ant-Man 2592 So he conspired against me and he voted me out of my own company.\n',
              "Ant-Man 2795 Hank, I'm a thief, all right?\n",
         

In [ ]:
for movie_name in movie_summaries:
  with open(f"{movie_name}_summary.txt", "w") as f:
    line_list = [lines.replace(movie_name, '') for lines in movie_summaries[movie_name]]
    f.writelines(line_list)

collate into one file for each movie

evaluation metric - after mapping back to original movie text

clean up somewhat

abstractive model

finish cleaning up

In [ ]:
args = Namespace(accum_count=2, alpha=0.6, batch_size=1, beam_size=5, bert_data_path='../PreSumm/split_bert_data/', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../PreSumm/logs/test_distilbert', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=150, max_pos=512, max_tgt_len=140, min_length=15, mode='test', model_path='./models/', optim='adam', other_bert='distilbert', param_init=0, param_init_glorot=True, recall_eval=False, report_every=10, report_rouge=False, result_path='./split_extractive_results/', save_checkpoint_steps=5, seed=666, sep_optim=False, share_emb=False, task='ext', temp_dir='../temp', test_all=False, test_batch_size=5, test_from='./models/model_step_5000.pt', test_start_from=-1, train_from='', train_steps=1000, use_bert_emb=False, use_interval=True, visible_gpus='-1', warmup_steps=1, warmup_steps_bert=8000, warmup_steps_dec=8000, world_size=1)


In [ ]:
args.test_from

'./models/model_step_5000.pt'

In [ ]:
checkpoint["model"].keys()

odict_keys(['bert.model.embeddings.word_embeddings.weight', 'bert.model.embeddings.position_embeddings.weight', 'bert.model.embeddings.LayerNorm.weight', 'bert.model.embeddings.LayerNorm.bias', 'bert.model.transformer.layer.0.attention.q_lin.weight', 'bert.model.transformer.layer.0.attention.q_lin.bias', 'bert.model.transformer.layer.0.attention.k_lin.weight', 'bert.model.transformer.layer.0.attention.k_lin.bias', 'bert.model.transformer.layer.0.attention.v_lin.weight', 'bert.model.transformer.layer.0.attention.v_lin.bias', 'bert.model.transformer.layer.0.attention.out_lin.weight', 'bert.model.transformer.layer.0.attention.out_lin.bias', 'bert.model.transformer.layer.0.sa_layer_norm.weight', 'bert.model.transformer.layer.0.sa_layer_norm.bias', 'bert.model.transformer.layer.0.ffn.lin1.weight', 'bert.model.transformer.layer.0.ffn.lin1.bias', 'bert.model.transformer.layer.0.ffn.lin2.weight', 'bert.model.transformer.layer.0.ffn.lin2.bias', 'bert.model.transformer.layer.0.output_layer_norm.

In [ ]:
dataset = torch.load("../PreSumm/split_bert_data/.test.0.bert.pt")

In [ ]:
dataset[0]["movie_name"]

'thorragnarok679alittlelessweaselly,lessgreasy,maybe.'

In [ ]:
dataset[0].keys()

dict_keys(['src', 'tgt', 'src_sent_labels', 'segs', 'clss', 'src_txt', 'tgt_txt', 'movie_name'])

In [ ]:
test = "thorragnarok679alittlelessweaselly,lessgreasy,maybe."
re.split('\d+', test)[0]

'thorragnarok'

In [ ]:
def get_corpus(filepath):
    movie_df = pd.read_csv(filepath)
    display(movie_df.head())
    display(movie_df.tail(), movie_df.shape)
    return movie_df

dataframe = get_corpus("corpus_merged.csv")
print("Successfully loaded the corpus data!")

,label,movie,corpus_movie,corpus_trailer,num_sentence_movie,num_sentence_trailer,num_words_movie,num_words_trailer,num_letters_movie,num_letters_trailer,merged_corpus
0,Marvel,Avengers Infinity War,"['This is the Asgardian', 'refugee vessel Stat...","['imagine a soldier the size of an insect', 't...",1723,26,9093,189,47892,926,"['This is the Asgardian', 'refugee vessel Stat..."
1,Marvel,Ant-Man and The Wasp,"['I still think about the night', 'your mother...","['So, how long have you been Ant-Man again?', ...",1929,24,10302,183,54559,935,"['I still think about the night', 'your mother..."
2,Marvel,Ant-Man,"['Stark!', ""He doesn't seem happy."", 'Hello, H...","['There are two types of', 'and those who do n...",1553,5,9528,31,50467,132,"['Stark!', ""He doesn't seem happy."", 'Hello, H..."
3,Marvel,Avengers Age of Ultron,"['Report to your stations immediately.', 'This...","[""I'm gonna show you something beautfiul"", 'Yo...",1902,9,10602,68,56711,323,"['Report to your stations immediately.', 'This..."
4,Marvel,Avengers Endgame,"[""Okay, hold on, don't shoot."", ""- You see whe...","['Oh No yeah hey go man no prob', 'star-lord w...",2160,23,11946,155,62876,847,"[""Okay, hold on, don't shoot."", ""- You see whe..."


,label,movie,corpus_movie,corpus_trailer,num_sentence_movie,num_sentence_trailer,num_words_movie,num_words_trailer,num_letters_movie,num_letters_trailer,merged_corpus
44,DC,The Dark Knight,"[""Three of a kind, let's do this. That's it? T...","[""He's in prison for putting Superman"", 'in th...",1826,39,13254,253,71327,1283,"[""Three of a kind, let's do this. That's it? T..."
45,DC,Watchmen,"['HAVE A GOOD TIME', 'Wrong, as usual.', ""The ...","[""don't do the circles please this mark"", 'wil...",2212,24,14317,178,81105,899,"['HAVE A GOOD TIME', 'Wrong, as usual.', ""The ..."
46,DC,The Dark Knight Rises,"['I knew Harvey Dent.', 'I was his friend.', '...",['Arthur does it help they have someone to tal...,1777,12,12609,106,67521,521,"['I knew Harvey Dent.', 'I was his friend.', '..."
47,DC,Wonder Woman 1984,"[' Some days, my childhood feels so very far a...","[""you've been gone a long time where did"", ""yo...",1893,20,10132,151,52365,745,"[' Some days, my childhood feels so very far a..."
48,DC,Wonder Woman,"['I used to want to save the world.', 'This be...","[""my life hasn't been what you probably"", 'thi...",1547,13,8750,94,46283,448,"['I used to want to save the world.', 'This be..."


(49, 11)

Successfully loaded the corpus data!


## Preprocessing Data

In [ ]:
def process_text(text):
  processed_text = re.sub("\'(,\s+)\'", '\n', text)
  processed_text = re.sub("[\[\]]", '', processed_text)
  return processed_text

In [ ]:
os.chdir("../PreSumm/src")

In [ ]:
from models.model_builder import ExtSummarizer
model = ExtSummarizer(args, device, checkpoint)
model.eval()

NameError: ignored

In [ ]:
!cat ./split_extractive_results/_step5000.gold

1706 wait ! 1710 he 's mine . 1711 so if you want him , you go through me . 1714 but we 've already got him . 1718 all right , then . i guess i go through you . 1721 - - more food . 1728 1732 1738 1745 1749 1751 1754 1757 - - thank you . 1761 1792 1794 scrapper 142 . i need clearance and an audience with the boss . 1798 i 've got something special . 1800 hey ! where are you taking me ? 1804 answer me ! 1806 hey ! 1807 i am thor , son of odin . i need to get back to asgard . 1811 - many apologies , your majesty . - 1813 1841 it 's come to my attention that you do n't know who i am . 1852 i am hela , odin 's firstborn ... 1856 the commander of the legions of asgard ... 1858 the rightful heir to the throne , 1860 and the goddess of death . 1866 my father is dead . 1869 as are the princes . you 're welcome . 1872 we were once the seat 1873 of absolute power in the cosmos . 1875 our supremacy was unchallenged . 1878 yet odin stopped at nine realms . 1881 our destiny is to rule over all othe

### Step 1.2: Download encoder
For this project we will be using an NLP encoder developed by stanford

In [ ]:
import site; site.getsitepackages()

['/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/lib/python3.7/dist-packages']

In [ ]:
!ls /usr/local/lib/python3.7/dist-packages/pyrouge

__init__.py  __pycache__  Rouge155.py  test.py	tests  utils


In [ ]:
!wget https://nlp.stanford.edu/software/stanford-corenlp-latest.zip

--2021-11-12 03:35:30--  https://nlp.stanford.edu/software/stanford-corenlp-latest.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-latest.zip [following]
--2021-11-12 03:35:30--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-latest.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 505031546 (482M) [application/zip]
Saving to: ‘stanford-corenlp-latest.zip.1’

stanford-corenlp-la  37%[======>             ] 181.57M  5.07MB/s    eta 53s    ^C


In [ ]:
!unzip stanford-corenlp-latest.zip

Go into the uziped file and maually unzip the file stanford-corenlp-4.3.1.jar

In [ ]:
!pip install -r ../PreSumm/requirements.txt # Install requirements for the package

     |████████████████████████████████| 1.6 MB 5.4 MB/s 
     |████████████████████████████████| 20.3 MB 1.4 MB/s 
     |████████████████████████████████| 60 kB 6.6 MB/s 
     |████████████████████████████████| 660 kB 36.6 MB/s 
     |████████████████████████████████| 190 kB 46.0 MB/s 
     |████████████████████████████████| 676.9 MB 4.1 kB/s 
     |████████████████████████████████| 895 kB 40.0 MB/s 
     |████████████████████████████████| 5.6 MB 34.9 MB/s 
     |████████████████████████████████| 1.2 MB 41.9 MB/s 
  Created wheel for multiprocess: filename=multiprocess-0.70.9-py3-none-any.whl size=108058 sha256=879c27d1f72e68e5981a28f8c022fc998baa82b1b292f51c0140d79709e3aeef
  Stored in directory: /root/.cache/pip/wheels/83/2b/b4/50d7cae5b9069434454fd36da009832592af4fff58b51db8d9
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191620 sha256=2d48975b8e01265e8a3ca09a37fe91c5edfc9d2a6902dea19644a9c37cd80fde
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/

In [ ]:
!pip install pytorch_transformers

     |████████████████████████████████| 176 kB 5.4 MB/s 
     |████████████████████████████████| 131 kB 43.0 MB/s 
     |████████████████████████████████| 8.1 MB 36.4 MB/s 
     |████████████████████████████████| 79 kB 6.8 MB/s 
     |████████████████████████████████| 138 kB 35.4 MB/s 
     |████████████████████████████████| 127 kB 45.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires numpy>=1.18.5, but you have numpy 1.17.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Step 1.3: Cloning the pretraining code

In [ ]:
!git clone https://github.com/chriskhanhtran/PreSumm.git

Cloning into 'PreSumm'...
remote: Enumerating objects: 467, done.
remote: Total 467 (delta 0), reused 0 (delta 0), pack-reused 467
Receiving objects: 100% (467/467), 13.13 MiB | 2.91 MiB/s, done.
Resolving deltas: 100% (285/285), done.
Checking out files: 100% (90/90), done.


### Step 1.4: Tokenize Data

In [ ]:
def get_corpus(filepath):
    movie_df = pd.read_csv(filepath)
    display(movie_df.head())
    display(movie_df.tail(), movie_df.shape)
    return movie_df

dataframe = get_corpus("corpus_merged.csv")
print("Successfully loaded the corpus data!")

,label,movie,corpus_movie,corpus_trailer,num_sentence_movie,num_sentence_trailer,num_words_movie,num_words_trailer,num_letters_movie,num_letters_trailer,merged_corpus
0,Marvel,Avengers Infinity War,"['This is the Asgardian', 'refugee vessel Stat...","['imagine a soldier the size of an insect', 't...",1723,26,9093,189,47892,926,"['This is the Asgardian', 'refugee vessel Stat..."
1,Marvel,Ant-Man and The Wasp,"['I still think about the night', 'your mother...","['So, how long have you been Ant-Man again?', ...",1929,24,10302,183,54559,935,"['I still think about the night', 'your mother..."
2,Marvel,Ant-Man,"['Stark!', ""He doesn't seem happy."", 'Hello, H...","['There are two types of', 'and those who do n...",1553,5,9528,31,50467,132,"['Stark!', ""He doesn't seem happy."", 'Hello, H..."
3,Marvel,Avengers Age of Ultron,"['Report to your stations immediately.', 'This...","[""I'm gonna show you something beautfiul"", 'Yo...",1902,9,10602,68,56711,323,"['Report to your stations immediately.', 'This..."
4,Marvel,Avengers Endgame,"[""Okay, hold on, don't shoot."", ""- You see whe...","['Oh No yeah hey go man no prob', 'star-lord w...",2160,23,11946,155,62876,847,"[""Okay, hold on, don't shoot."", ""- You see whe..."


,label,movie,corpus_movie,corpus_trailer,num_sentence_movie,num_sentence_trailer,num_words_movie,num_words_trailer,num_letters_movie,num_letters_trailer,merged_corpus
44,DC,The Dark Knight,"[""Three of a kind, let's do this. That's it? T...","[""He's in prison for putting Superman"", 'in th...",1826,39,13254,253,71327,1283,"[""Three of a kind, let's do this. That's it? T..."
45,DC,Watchmen,"['HAVE A GOOD TIME', 'Wrong, as usual.', ""The ...","[""don't do the circles please this mark"", 'wil...",2212,24,14317,178,81105,899,"['HAVE A GOOD TIME', 'Wrong, as usual.', ""The ..."
46,DC,The Dark Knight Rises,"['I knew Harvey Dent.', 'I was his friend.', '...",['Arthur does it help they have someone to tal...,1777,12,12609,106,67521,521,"['I knew Harvey Dent.', 'I was his friend.', '..."
47,DC,Wonder Woman 1984,"[' Some days, my childhood feels so very far a...","[""you've been gone a long time where did"", ""yo...",1893,20,10132,151,52365,745,"[' Some days, my childhood feels so very far a..."
48,DC,Wonder Woman,"['I used to want to save the world.', 'This be...","[""my life hasn't been what you probably"", 'thi...",1547,13,8750,94,46283,448,"['I used to want to save the world.', 'This be..."


(49, 11)

Successfully loaded the corpus data!


In [ ]:
train_df, test_df = train_test_split(dataframe, test_size=0.2, random_state=1)
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=1)

def process_text(text):
  processed_text = re.sub("\'(,\s+)\'", '\n', text)
  processed_text = re.sub("[\[\]]", '', processed_text)
  return processed_text

train_df["merged_corpus"] = train_df["merged_corpus"].apply(lambda x: process_text(x))
test_df["merged_corpus"] = test_df["merged_corpus"].apply(lambda x: process_text(x))
val_df["merged_corpus"] = val_df["merged_corpus"].apply(lambda x: process_text(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
train_df.head()

,label,movie,corpus_movie,corpus_trailer,num_sentence_movie,num_sentence_trailer,num_words_movie,num_words_trailer,num_letters_movie,num_letters_trailer,merged_corpus
15,Marvel,Spider-Man Homecoming,"['Things are never gonna be the same now.', 'I...","['Could we pick up now where we left off', 'Ca...",1939,17,13035,119,69265,617,'Things are never gonna be the same now.\nI me...
14,Marvel,Deadpool,"['Kinda lonesome back here.', 'Okay, um, just,...","['coming up on the drop zone cap you do', 'any...",1574,25,8514,180,46880,889,"'Kinda lonesome back here.\nOkay, um, just, I ..."
16,Marvel,Guardians of the Galaxy,"['Peter.', 'Your momma wants to speak with you...","['after all this time now you come to', '- moc...",1391,16,8103,119,42791,562,'Peter.\nYour momma wants to speak with you.\n...
8,Marvel,Iron Man 3,"['A famous man once said,', '""we create our ow...","['There was an idea.', 'A group of remarkable ...",1815,13,11390,72,60155,398,"'A famous man once said,\n""we create our own d..."
5,Marvel,Black Panther,"['Baba...', 'Yes, my son.', 'Tell me a story.'...","['YELENA BELOVA It was real to me.', '...you w...",1574,6,7958,35,42467,200,"'Baba...\nYes, my son.\nTell me a story.\nWhic..."


In [ ]:
os.chdir("./PreSumm/corpus")

In [ ]:
split_dict = {"train": train_df, "val": val_df, "test": test_df}
for split, df in split_dict.items():
  # dir_path = os.path.join(os.getcwd(), split)  
  # os.chdir(dir_path)  # navigate to correct subdirectory (train, test, or val)
  for i in df.index:
    script = df["merged_corpus"][i]
    movie_name = df["movie"][i]
    with open(f"{split}_{movie_name}.txt", "w") as f:  # write script to text file
      f.write(script)

In [ ]:
# %env CLASSPATH=./stanford-corenlp-4.3.1/stanford-corenlp-4.3.1.jar # Set the proper env

env: CLASSPATH=./stanford-corenlp-4.3.1/stanford-corenlp-4.3.1.jar # Set the proper env


### Step 1.5: Format to Simpler Json Files

In [ ]:
# Need to update
!python ../PreSumm/src/preprocess.py -mode format_to_lines -raw_path ../PreSumm/tokenized_corpus/ -save_path ../PreSumm/json_data/ -n_cpus 1 -use_bert_basic_tokenizer false

../PreSumm/tokenized_corpus/train_Spider-Man Homecoming.txt.json
../PreSumm/tokenized_corpus/train_Deadpool.txt.json
../PreSumm/tokenized_corpus/train_Guardians of the Galaxy.txt.json
../PreSumm/tokenized_corpus/train_Iron Man 3.txt.json
../PreSumm/tokenized_corpus/train_Black Panther.txt.json
../PreSumm/tokenized_corpus/train_The Dark Knight Rises.txt.json
../PreSumm/tokenized_corpus/train_Black Widow.txt.json
../PreSumm/tokenized_corpus/train_Captain America The Winter Soldier.txt.json
../PreSumm/tokenized_corpus/train_Man of Steel.txt.json
../PreSumm/tokenized_corpus/train_Iron Man.txt.json
../PreSumm/tokenized_corpus/train_Logan.txt.json
../PreSumm/tokenized_corpus/train_Thor.txt.json
../PreSumm/tokenized_corpus/train_Captain America The First Avenger.txt.json
../PreSumm/tokenized_corpus/train_Wonder Woman 1984.txt.json
../PreSumm/tokenized_corpus/train_Deadpool 2.txt.json
../PreSumm/tokenized_corpus/train_Batman v Superman Dawn of Justice.txt.json
../PreSumm/tokenized_corpus/train

### Step 1.6: Format to PyTorch Files

In [ ]:
os.listdir("../PreSumm/json_data")

['.train.0.json', '.valid.0.json', '.test.0.json']

In [ ]:
# Need to update
!python ../PreSumm/src/preprocess.py -mode format_to_bert -raw_path ../PreSumm/json_data/ -save_path ../PreSumm/bert_data/ -lower -n_cpus 1 -log_file ../PreSumm/logs/preprocess.log

[('train', '../PreSumm/json_data/.train.0.json', Namespace(dataset='', log_file='../PreSumm/logs/preprocess.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='../PreSumm/json_data/', save_path='../PreSumm/bert_data/', select_mode='greedy', shard_size=2000, use_bert_basic_tokenizer=False), '../PreSumm/bert_data/.train.0.bert.pt'), ('train', '../PreSumm/json_data/.valid.0.json', Namespace(dataset='', log_file='../PreSumm/logs/preprocess.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='../PreSumm/json_data/', save_path='../PreSumm/bert_data/', select_mode='greedy', shard_size=2000, use_bert_basic_tokeniz

### Misc cells I was afraid to delete

In [ ]:
!touch ../tokenized_corpus/tokenized_corpus.csv

In [ ]:
!./cnn/stories/

/bin/bash: ./cnn/stories/: Is a directory


In [ ]:
!cp ../../corpus_raw/Marvel/trailer_corpus/*.txt ../corpus/

cp: cannot stat '../../corpus_raw/Marvel/trailer_corpus/*.txt': No such file or directory


In [ ]:
!wget https://drive.google.com/file/d/0BwmD_VLjROrfTHk4NFg2SndKcjQ/view?resourcekey=0-plDHB0-3Jgnal3MfHKQS1w

--2021-11-11 23:58:01--  https://drive.google.com/file/d/0BwmD_VLjROrfTHk4NFg2SndKcjQ/view?resourcekey=0-plDHB0-3Jgnal3MfHKQS1w
Resolving drive.google.com (drive.google.com)... 142.251.31.102, 142.251.31.139, 142.251.31.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.31.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?resourcekey=0-plDHB0-3Jgnal3MfHKQS1w’

view?resourcekey=0-     [ <=>                ]  65.36K  --.-KB/s    in 0.08s   

2021-11-11 23:58:01 (828 KB/s) - ‘view?resourcekey=0-plDHB0-3Jgnal3MfHKQS1w’ saved [66932]



## Step 2: Training Model

In [ ]:
!python ../PreSumm/src/train.py -task ext -mode train -bert_data_path ../PreSumm/bert_data/ -ext_dropout 0.1 -lr 2e-2 -visible_gpus 0 -report_every 10 -save_checkpoint_steps 500 -batch_size 5 -train_steps 5000 -accum_count 2 -log_file ../PreSumm/logs/distilbert_2 -use_interval true -warmup_steps 1 -max_pos 512 -other_bert distilbert


In [ ]:
!touch ./results/step5000.candidate

In [ ]:
!touch ./results/step5000.gold